In [92]:
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
import random
pd.set_option('display.max_columns', 500)

data = pd.read_csv('Участники anonimized.csv') #Файл с общими данными о всех участниках. Его обработка

def ClearNans(data):
  for i in data.columns:  #Удаляем столбцы, где больше 40% пропусков
    if data[i].isna().sum() / len(data[i]) > 0.4:
      data = data.drop(i, axis = 1)
  return data

data = ClearNans(data)

#обработка дат

def date_parser(i, column, nans):
  s = data[column][i]
  s = str(s)
  try:
    s = int(s)
    if s > 2023:
      return 0
    else:
      return s
  except:
    pass
  if nans[i] == True:
    return 0
  if len(s) == 10:
    try:
      return int(s[0 : 4])
    except:
      try:
        return int(s[-4:])
      except:
        pass
  try:
    if len(s) == 4:
      return int(s)
  except:
    return 0
  if len(s) == 1:
    return 0
  if s[-3]=='.' or s[-3]=='.' or s[-3] == ',' or s[-3] == ',' or s[-3] == '-':
    if int(s[-2:]) < 30:
      return int(s[-2:]) + 2000
    else:
      return int(s[-2:]) + 1900
  try:
    if s[-5] == '.' or s[-5] == '.' or s[-5] == ' ' or s[-5] == ',' or s[-5] == ',':
      return int(s[-4:])
  except:
    pass
  if 'г' in s:
    return int(s[:4])
  try:
    s = int(s.split('.')[0])
    return s
  except:
    pass
  return 0

#Кодирование категориальных переменных

def date_coder(df, column):
  nans = df[column].isna()
  df[column] = [date_parser(i, column, nans) for i in range(len(df[column]))]
  df[column] = [int(i) for i in df[column]]
  df[column].replace(to_replace = 0, value = df[df[column] != 0][column].mean() , inplace = True)
  return df

data = date_coder(data, 'Начало трудового стажа')
data = date_coder(data, 'Дата рождения')

data['Роль в мероприятии'] = data['Роль в мероприятии'].map({'Участник не в оценке': 0, 'Участник мероприятия': 1})
data = data.astype({"Роль в мероприятии": "Int64"})

def coder(df, column, k):
  ind = df[column].value_counts().index
  vals = list(df[column].value_counts())
 
  j = 0
  for i in range(len(ind)):
    df.loc[df[column] == ind[i], column] = j
    if vals[i] > k:
      j += 1
  df = df.fillna(j)
  return df
 
data = coder(data, 'Должность', 1)
data = coder(data, 'Место работы', 2)

napr = data['Список компетенций'].unique()

label_encoder = preprocessing.LabelEncoder() 
label_encoder.fit(data['Список компетенций']) 
data['Список компетенций'] = label_encoder.transform(data['Список компетенций'])

def ParseFio(data):
  data['ФИО'] = [int(i.split('_')[1]) if '_' in i else 9999999 for i in data['ФИО']]
  return data

data = ParseFio(data)

'''Обработка датасетов по каждому из направлений чемпионата.
В каждом из представленных отчётов по направлениям представлено большое количество признаков.
Для более успешной кластеризации необходимо понизить размерность. Используем метод PCA.'''

df_mas_ = []  #список из файлов
j = 0
for i in napr:
  df_mas_.append(pd.read_csv(i[:len(i) - 2] + '.csv', sep = ','))

def convert_comma_to_float(df):
    for col in df.columns:
        if df[col].dtype == 'O':  
            df[col] = df[col].str.replace(',', '.') 
            df[col] = df[col].astype(float) 
    return df

def clean_data(df): 
    df.dropna(axis = 1, inplace = True)
    if 'Баллы, %' in df.columns:
      df['Баллы, %'] = df['Баллы, %'].str.replace('%', '')
    while(df.columns[0] != 'Баллы, ед.'):
      df = df.drop(df.columns[0], axis = 1)
    convert_comma_to_float(df)
    return df

def scale_and_PCA(df):
  scaler = StandardScaler()
  try:
    scaled_data = scaler.fit_transform(df)
  except:
    print(df)
  return scaled_data

def do_pca(df, num_com):
  scale = scale_and_PCA(clean_data(df))

  n_components = num_com
  pca = PCA(n_components = n_components)
  principal_components = pca.fit_transform(scale)

  variance_ratio = pca.explained_variance_ratio_
  return principal_components

i = 0
for df in df_mas_:
    if('ФИО участников' in df.columns):
      df['ФИО участников'] = df['ФИО участников'].str.split(';')
      df_new = df.explode('ФИО участников')
      df_new = df_new.rename(columns = {"ФИО участников": "ФИО"})
      df_new.index = [i for i in range(len(df_new))]
      df_mas_[i] = df_new
    i += 1

arr_ = []
for df in df_mas_:
  try:
    arr_.append(do_pca(df, 7))
  except:
    print(df)

arr_2 = []
i = 0
for df in df_mas_:
  columns = ['feature1', 'feature2', 'feature3', 'feature4', 'feature5', 'feature6', 'feature7']
  df_smth = pd.DataFrame(arr_[i], columns = columns)
  try:
    fio = df['ФИО']
  except:
    print(df)
  res = pd.concat([fio, df_smth], axis = 1)
  arr_2.append(res)
  i += 1

new_df = pd.concat(arr_2, axis = 0)

new_df = ParseFio(new_df)
new_df = new_df.sort_values(by = 'ФИО')
new_df.index = [i for i in range(len(new_df))]

feat = new_df

dr = []
for i in range(len(feat.ФИО)):
  if feat.ФИО[i] == 9999999:
    dr.append(i)

feat = feat.drop(dr, axis = 0)
feat = feat.sort_values(by = 'ФИО')
feat.reset_index(drop = True , inplace = True)

dr = []
for i in range(len(data)):
  if data.loc[i].ФИО not in list(feat.ФИО):
    dr.append(i)

data = data.drop(dr, axis = 0)
data = data.sort_values(by = 'ФИО')
data.reset_index(drop = True , inplace = True)

feat = feat.drop('ФИО', axis = 1)

data = pd.merge(data, feat, left_index = True, right_index = True, how = 'left')

In [93]:
data

,ФИО,Пол,Роль в мероприятии,Список компетенций,Должность,Начало трудового стажа,Место работы,Дата рождения,feature1,feature2,feature3,feature4,feature5,feature6,feature7
0,0,0,1,36,83,1993.000000,36,1975.000000,-0.168683,3.168650,9.343456,-1.568493,-2.175098e+00,6.181112e-01,-2.373866e+00
1,1,1,1,34,2,2000.000000,44,1977.000000,12.172361,1.909320,-4.893528,0.360889,-4.700776e-01,1.192233e-01,-2.170459e+00
2,2,0,1,26,47,2011.466546,44,1988.679208,8.590002,-0.874525,1.245231,-4.127800,-3.353448e+00,5.793364e-01,-4.292498e-01
3,3,0,1,7,48,2005.000000,44,1986.000000,5.300281,-3.444928,4.555605,-8.520903,-4.308845e+00,4.725987e+00,-1.031182e+01
4,4,0,1,5,81,2011.466546,44,1988.679208,-5.841847,9.258686,1.653345,0.571360,3.241864e-01,1.459440e-15,6.003816e-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705,708,0,1,2,83,2011.466546,44,1988.679208,0.395246,8.535900,0.970737,-3.451364,7.959877e-16,-1.106870e-16,1.961495e-16
706,709,1,1,34,83,2013.000000,8,1990.000000,-8.047304,-6.361686,-2.613589,-7.141867,-4.997863e+00,1.535965e-01,-5.303137e+00
707,710,1,1,1,59,2016.000000,44,1993.000000,2.422774,-0.594510,-4.515643,-2.628545,-3.862591e+00,1.524534e+00,-6.434879e-01
708,711,0,1,32,40,2011.466546,2,1988.679208,-1.984648,-10.260374,-6.197178,1.123215,-7.301041e-01,7.400600e+00,-3.752828e-01


In [94]:
feat

,feature1,feature2,feature3,feature4,feature5,feature6,feature7
0,-0.168683,3.168650,9.343456,-1.568493,-2.175098e+00,6.181112e-01,-2.373866e+00
1,12.172361,1.909320,-4.893528,0.360889,-4.700776e-01,1.192233e-01,-2.170459e+00
2,8.590002,-0.874525,1.245231,-4.127800,-3.353448e+00,5.793364e-01,-4.292498e-01
3,5.300281,-3.444928,4.555605,-8.520903,-4.308845e+00,4.725987e+00,-1.031182e+01
4,-5.841847,9.258686,1.653345,0.571360,3.241864e-01,1.459440e-15,6.003816e-17
...,...,...,...,...,...,...,...
705,0.395246,8.535900,0.970737,-3.451364,7.959877e-16,-1.106870e-16,1.961495e-16
706,-8.047304,-6.361686,-2.613589,-7.141867,-4.997863e+00,1.535965e-01,-5.303137e+00
707,2.422774,-0.594510,-4.515643,-2.628545,-3.862591e+00,1.524534e+00,-6.434879e-01
708,-1.984648,-10.260374,-6.197178,1.123215,-7.301041e-01,7.400600e+00,-3.752828e-01


In [95]:
data.to_csv('data.csv', index = False)
feat.to_csv('feat.csv', index = False)